In [11]:
from keras.layers import Bidirectional, Concatenate, Permute, Dot, Input, LSTM, Multiply
from keras.layers import RepeatVector, Dense, Activation, Lambda
from keras.optimizers import Adam
from keras.utils import to_categorical
from keras.models import load_model, Model
import keras.backend as K
import keras.preprocessing
import numpy as np
import pandas as pd
import re

In [3]:
data = pd.read_csv('../INPUT/goodreads_reviews_clean.csv')
data.head()

,review,grade,lang,score,sentiment
0,"The statue of Hadrian, the 14th Emperor of the...",it was amazing,en,4,2
1,It's hard to know what to think about this boo...,really liked it,en,3,2
2,"If you're into stuff like this, you can read t...",it was amazing,en,4,2
3,Has Kenzaburo Oe ever visited Japan? Ban this ...,it was amazing,en,4,2
4,Sadza (typical cornmeal staple food in Zimbabw...,it was amazing,en,4,2


Create vocabulary:


In [19]:
text = ''
for rev in data.review:
    text += ' ' + rev

Let's use keras tokenizer to preprocess the text:    

In [31]:
tokenizer = keras.preprocessing.text.Tokenizer(char_level=False)
tokenizer.fit_on_texts(data.review)

In [38]:
tokenizer.sequences_to_texts([[11, 136, 8, 1, 70, 213]])

['this author is the very best']

In [34]:
tokenizer.texts_to_sequences(["this author is the very best"])

[[11, 136, 8, 1, 70, 213]]

In [36]:
len(tokenizer.word_index)

110609

In [53]:
[encoded] = np.array(tokenizer.texts_to_sequences(data.review))

ValueError: too many values to unpack (expected 1)

<table>
<td> 
<img src="images/attn_model.png" style="width:500;height:500px;"> <br>
</td> 
<td> 
<img src="images/attn_mechanism.png" style="width:500;height:500px;"> <br>
</td> 
</table>
<caption><center> **Figure 1**: Neural machine translation with attention</center></caption>